# Mini	projet	3	:	Prédiction	de	la	structure	secondaire	avec	GOR

***

**PAQUET Michael (000410753) BA-INFO**

# Introduction

Dans le milieu biologique, les protéines peuvent être représentées sous différentes structures en fonction de nos besoins.
Celles-ci sont connues sont le nom de structure primaire resprésentant les protéines sous forme de chaine d'acides aminés, structure secondaire représentant la protéine sous forme d'α-hélices, de β-brins et de coils. Il existe également les structures tertiaires et quaternaire, mais elles ne feront pas l'objet de cette étude-ci.

Afin d'obtenir diverses informations sur une protéine donnée, il est intéressant d'en connaitre sa structure secondaire. C'est pourquoi nous allons parler de methodes nous permettant de prédire cette structure avec un taux d'exactitude relativement élevé. 

Au cours de ce projet-ci, l'algorithme de GOR III a été choisi afin de répondre à notre problématique. Celui-ci permet théoriquement d'avoir un taux d'exactitude nommé Q3 de 63%. Nous allons donc implémenter GOR III, et sur base de données, allons obtenir des résultats dont nous pourrons finalement discuter. 

Voici brièvement comment fonctionne GOR III :
sur base d'un jeu de donnée, l'algorithme créera un modèle constitué d'informations individuelles et d'informations par pair.
- Les informations individuelles sont des propensions représentant "la chaince qu'un acide aminé R soit de structure S".
- Les informations par pair indiquent le nombre de fois qu'on a croisé un acide aminé Y dans le voisinage de R. 

Ainsi, lorsque nous parcourrons une protéine donnée, nous ferons appel à notre modèle calculé précédemment afin d'en prédire la structure secondaire. 

# Implémentation

## Parsing des données

Avant de commencer à réfléchir sur la façon d'implémenter GOR III, il nous faut tout d'abord collecter de façon intélligente les informations dont nous disposons dans un jeu de donnée. Ce jeu de donnée est composé de 3713 protéines. 3000 d'entre elles serviront à construire notre modèle, tandis que les 713 autres seront soumis à notre algorithme. Afin de simplifier cette collection de donnée, les identifiants de chacune des protéines seront listées dans un fichier texte nommé "CATH_info". D'autre part, dans un autre dossier nommé "dssp", un fichier texte par protéine dont le nom en est l'identifiant sera présent. 

Au cours du parsing des données, nous allons donc parcourir CATH_info, et pour chaque identifiant nous parcourrons le fichier de la protéine lié à cet identifiant pour en retirer les informations nous étant utiles. 

In [12]:
def parsing(myFile):
    myfile = open(myFile, "r")
    for line in myfile:
        current = line.replace("\n", "").split(" ")
        ID = current[0][0:4] #ID de la protéine à parser
        chaine = current[0][4] #Chaine à regarder uniquement
        dsspFile = "dataset/dssp/" + ID + ".dssp"
        parsingDSSP(dsspFile, chaine)

__Parsing :__

A noter que dans CATH_info, deux informations sont présentes mais seule la première nous intéresse. 
L'information que nous collectons est composé de l'ID de la protéine, ainsi que la chaine à regarder. En effet, il existe des doublons, c'est pourquoi le fichier nous donne en plus de l'ID la chaine à regarder pour chaque protéine. 
Nous pouvons donc finalement parser le fichiers des protéines.

In [13]:
def parsingDSSP(myFile, chaine):
    myfile = open(myFile, "r")
    parsedFile = open("data.txt", "a")
    cList = ['p', 'q', 'n', 'd', 'l', 'e', 'b', 'f', 'o', 'c', 'k', 'r', 'a', 'm', 'h', 'j', 'g', 'i']
    dic = {"H" : "H", "G" : "H", "I" : "H", "E" : "E", "B" : "E", "T" : "C", "C" : "C", "S" : "C", " ": "C"}
    notAA = ["X", "B", "Z"]
    sequence = ""
    structure = ""
    reading = False
    parsedFile.write("> identifier|protein name|organism\n")
    for line in myfile:
        if line[0:3] == "  #":
            reading = True
        elif reading:
            if line[11] == chaine:
                AA = line[13]
                if AA in cList:
                    AA = "C"
                if AA not in notAA:
                    sequence += AA
                    structure += dic[line[16]]
    parsedFile.write(sequence)
    parsedFile.write("\n")
    parsedFile.write(structure)
    parsedFile.write("\n")

__parsingDSSP :__

Maintenant qu'on est en sein de notre fichier protéine, il nous faut collecter les informations pertinentes. 
Dans ce fichier, les données sont structurée de cette façon : 
- __Numéro de l'AA - Résidu - AA - Structure__, où "-" représente un espace. 

D'autres informations sont également présentes, mais sont inutiles dans le cadre de notre projet. Seuls l'acide aminé et sa structure nous intéressent ici.

A noter que les lettres minuscules sencés représenter un AA seront remplacées par une Cystéine, ce à quoi sert __cList__.
__dico__ nous permet d'associer la structure collectée aux 3 structures principales. En effet, les fichiers présentent plusieurs structures secondaires mais seuls les hélices, les brins et les coils nous intéressent ici. 
Enfin, __notAA__ nous permet de ne pas prendre en compte certaine lettres ne représentant pas un AA. 

Tous ces objets nous permettent au final de collecter l'AA et sa structure secondaire associée, afin de les garder dans un fichier nommé "data.txt". Ce fichier sera construit comme tel : pour chaque acide aminé : 
- \> identifiant
- chaine d'AA
- structures secondaires associées à chaque AA

Voici à titre d'exemple une illustration de ce que donne le fichier "data.txt" pour la première protéine listée dans CATH_info. 

In [19]:
def main():
    try:
        open("data.txt", "r")
        print("Les données sont déjà collectées\n")
    except:
        print("Parsing des données en cours")
        myFile = "dataset/CATH_info.txt"
        parsing(myFile)
    printedFile = open("data.txt", "r")
    i = 0
    for line in printedFile:
        print(line, end = "")
        i+= 1
        if i == 3:
            break
            
main()

Les données sont déjà collectées

> identifier|protein name|organism
RTDCYGNVNRIDTTGASCKTAKPEGLSYCGVSASKKIAERDLQAMDRYKTIIKKVGEKLCVEPAVIAGIISRESHAGKVLKNGWGDRGNGFGLMQVDKRSHKPQGTWNGEVHITQGTTILINFIKTIQKKFPSWTKDQQLKGGISAYNAGAGNVRSYARMDIGTTHDDYANDVVARAQYYKQHGY
CCCCCCCHHHCCCCCECHHHHHHHCCCCCEHHHHHHHHHHCHHHHHCCHHHHHHHHHHHCCCHHHHHHHHHHHHCCCCCCECCECCCCCEECCCCEECCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHCCCCCHHHHHHHHHHHHHHCHHHCCCCCCCCCCCHHHCHHHHHHHHHHHHHHCCC


## GOR III

Maintenant que nous avons collecté toutes les données nécessaires à la construction de notre modèle, il est temps de réfléchir à l'implémentation de GOR III.

Nous en avons parlé plus tôt, GOR III se base sur deux informations : information individuelle et par pair.

__Informations individuelles :__

Afin d'avoir les propenstions individuelles, nous parcourrons notre fichier "data" et pour chaque protéine, nous allons compter le nombre de fois qu'un acide aminé __R__ est de structure __S__.

__Informations par pair :__

Lors du même parcours que celui des informations individuelles, un parcours du voisinage de l'acide aminé __R__ est effectué. La fenêtre que nous parcourrons sera d'une taille 16 sans compter __R__.

$$\cdot \cdot[ \cdot \cdot \cdot \cdot \cdot \cdot \cdot \cdot R \cdot \cdot \cdot \cdot \cdot \cdot \cdot \cdot]\cdot \cdot$$

Ainsi, si nous croisons l'acide aminé __Y__ lors du parcours de la fenêtre de __R__, et que __R__ est de structure __S__ :

$$\cdot \cdot[ Y \cdot \cdot \cdot \cdot \cdot \cdot \cdot R \cdot \cdot \cdot \cdot \cdot \cdot \cdot \cdot]\cdot \cdot$$

Alors on incrémente un compteur qui nous informe qu'__Y__ a été rencontré dans le voisinage de __R__ avec une structure __S__. Cette opération est itérée pour chaque acide aminé dans la fenêtre de __R__.

__Propensions :__

Une fois les parcours terminés et nos compteurs à jour, ils nous faut calculer la propension à partir de ceux-ci. Voici la formule mathématique nous le permettant : 

- Probabilité qu'un acide aminé R adopte la structure S :
$$I(\Delta S; R) = I(S; R) - I(n-S; R) = \log{\frac{f_{S, R}}{f_{n-S, R}}} + \log{\frac{f_{n-S}}{f_{S}}}$$

avec __Fs__ la fréquence de la structure, __Fs,r__ la fréquence de l'acide aminé __R__ dans la structure __S__ et __n-S__ les autres structures que __S__.


- Probabilité qu'un couple d'acide aminé Rj et Rj+m apparaissent avec pour structure S:
$$I(\Delta S_{j}; R_{j+m}|R_{j}) = \log{\frac{f_{S_{j}, R_{j+m}, R_{j}}}{f_{n-S_{j}, R_{j+m}, R_{j}}}} + \log{\frac{f_{n-S_{j}, R_{j}}}{f_{S_{j}, R_{j}}}}$$

avec __Fsj,rj+m,rj__ la fréquence d'observation de __Rj+m__ et __Rj__ dans une structure __S__ pour -8 <= __m__ <= 8 et __m__ != 0.

# Résultats

# Observation & discussion

# Conclusion